# Compute

> Compute modules for benchmarking

In [ ]:
#| default_exp compute

In [ ]:
#| export
from __future__ import annotations

import warnings
from dataclasses import dataclass, asdict
from typing import Dict

import torch
import torch.nn as nn

try:
    from thop import profile as _thop_profile
except ImportError:
    _thop_profile = None

try:
    from torchprofile import profile_macs as _profile_macs
except ImportError:
    _profile_macs = None

In [ ]:
#| export
@dataclass(slots=True)
class ComputeMetrics:
    """MACs (in **millions**) and parameter count (also millions)."""

    macs_m: float | str  # may be "*" if unavailable
    params_m: float

    def as_dict(self) -> Dict[str, float | str]:
        return asdict(self)


#| export
def compute_compute(model: nn.Module, dummy_input: torch.Tensor) -> ComputeMetrics:
    """Return **ComputeMetrics** for a single forward pass."""

    params_m = round(
        sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6, 3
    )

    if _thop_profile is not None:
        try:
            mac_raw, _ = _thop_profile(model, inputs=(dummy_input,))
            macs_m: float | str = round(mac_raw / 1e6, 3)
        except Exception as e:  # pragma: no cover
            warnings.warn(f"thop failed: {e}")
            macs_m = "*"
    elif _profile_macs is not None:
        try:
            macs_m = round(_profile_macs(model, dummy_input) / 1e6, 3)
        except Exception as e:  # pragma: no cover
            warnings.warn(f"torchprofile failed: {e}")
            macs_m = "*"
    else:
        warnings.warn("No MAC‑counting backend available – skipping MACs")
        macs_m = "*"

    return ComputeMetrics(macs_m=macs_m, params_m=params_m)